# 1: Import essential libraries

In [2]:
import sys
sys.path.append('/home/quangminh/Documents/code/Python/kfot/notebooks')

In [3]:
%cd ..

/home/quangminh/Documents/code/Python/kfot


In [5]:
from transformers import CLIPModel,CLIPProcessor
import torch
from PIL import Image
import ot
from random import randint

# Disable warnings
import warnings
warnings.filterwarnings('ignore')

from optimal_transport.experiments.vision_language.datasets import *

from tqdm import tqdm

/home/quangminh/anaconda3/envs/OT-CLIP/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 2: Embedding images and texts

In [4]:
# Model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [5]:
# # Define the transformations
# transform = transforms.Compose([
#     transforms.RandomHorizontalFlip(),   # Randomly flip the images horizontally for data augmentation
#     transforms.RandomCrop(32, padding=4), # Randomly crop the image with padding
#     transforms.ToTensor(),                # Convert PIL image to PyTorch Tensor
#     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)) # Normalize with mean and std
# ])

In [12]:
test_data = cifar10()

Files already downloaded and verified


In [7]:
n_classes = len(test_data.classes)
n_samples = len(test_data)  # total number of samples
res_img = torch.empty((n_samples, 512))  # Preallocate memory
res_text = torch.empty((n_samples, 512))
index = 0
for class_name in tqdm(test_data.classes):
    # Get batched data for the current class
    loader = test_data.get_batch_loader(class_name, batch_size=32)
    for batch in loader:
        images, labels, _ = batch
        # Preprocess images and text labels
        inputs = processor(text=labels, images=images, return_tensors='pt', padding=True)
        with torch.no_grad():
            outputs = model(**inputs)
            images_embeds = outputs.image_embeds  # embeddings for this batch
            text_embeds = outputs.text_embeds
        # Save the embeddings to the pre-allocated tensor
        batch_size = images_embeds.size(0)
        res_img[index:index + batch_size] = images_embeds
        res_text[index:index + batch_size] = text_embeds
        index += batch_size

100%|██████████| 10/10 [14:41<00:00, 88.15s/it]


In [8]:
with open('data/cifar10/embedded/cifar10-image-embeds.pt','wb') as f:
    torch.save(res_img,f)

In [9]:
with open('data/cifar10/embedded/cifar10-text-embeds.pt','wb') as f:
    torch.save(res_text,f)

In [6]:
# test 
img_embedding, text_embedding = torch.load('data/cifar10/embedded/cifar10-image-embeds.pt'), torch.load('data/cifar10/embedded/cifar10-text-embeds.pt')

In [7]:
print(img_embedding) 
print(text_embedding)

tensor([[ 0.0336, -0.0199, -0.0080,  ...,  0.0675, -0.0410, -0.0146],
        [ 0.0287,  0.0071,  0.0077,  ...,  0.0560,  0.0004,  0.0079],
        [-0.0079, -0.0167, -0.0181,  ...,  0.0759,  0.0036, -0.0121],
        ...,
        [ 0.0153, -0.0238, -0.0121,  ...,  0.0715,  0.0095,  0.0191],
        [ 0.0240, -0.0024, -0.0517,  ...,  0.0776, -0.0176,  0.0149],
        [-0.0143,  0.0002, -0.0143,  ...,  0.0400, -0.0037,  0.0150]])
tensor([[ 0.0058,  0.0009,  0.0031,  ...,  0.0072, -0.0021, -0.0193],
        [ 0.0058,  0.0009,  0.0031,  ...,  0.0072, -0.0021, -0.0193],
        [ 0.0058,  0.0009,  0.0031,  ...,  0.0072, -0.0021, -0.0193],
        ...,
        [ 0.0064,  0.0218, -0.0012,  ..., -0.0359, -0.0177,  0.0077],
        [ 0.0064,  0.0218, -0.0012,  ..., -0.0359, -0.0177,  0.0077],
        [ 0.0064,  0.0218, -0.0012,  ..., -0.0359, -0.0177,  0.0077]])


# 3: Test zero-shot inference

In [18]:
from sklearn.metrics import accuracy_score
from optimal_transport.adapters.info import InfoOT
from optimal_transport.adapters.emd import EMD
from optimal_transport.adapters.entropic import EntropicOT
from optimal_transport.adapters.low_rank import LrSinkhornOT
from optimal_transport.adapters.kpg.kp import KPGRLKP
from optimal_transport.adapters.factored.fot import FOT
from optimal_transport.adapters.factored.kfot import KFOT
from optimal_transport.adapters.factored.lot import LOT
import torch.nn.functional as F

In [ ]:
# Fused-Gromov softmax
def fgwi_softmax(image_features, text_features, a1, a2, reg, n):
    """
    image_features: image features
    text_features: text features
    a1: regularization term for scaling the KL divergence term 1
    a2: regularization term for scaling the KL divergence term 2
    reg: regularization parameter > 0
    n: number of iterations for adjusting the structural cost matrix
    """

    # KL term 1: compute the self-similarity matrix for image features
    sigma_1 = image_features @ image_features.t()
    # KL term 2: compute the self-similarity matrix for text features
    sigma_2 = text_features @ text_features.t()

    # init cost matrix
    C = 1.0 - image_features @ text_features.t()
    # init transport plan
    P = F.softmax(-C / reg)

    for _ in tqdm(range(n)):
        C = C - a1 * sigma_1 @ P @ sigma_2 * a2
        P = F.softmax(-C/ reg)
    return P

transport_plan = fgwi_softmax(img_embedding, text_embedding, 1e-4,1e-4,0.05,100)
res = [test_data.targets[x] for x in transport_plan.argmax(1)]

print(accuracy_score(test_data.targets,res))

100%|██████████| 1/1 [00:20<00:00, 20.72s/it]


TypeError: 'builtin_function_or_method' object is not subscriptable

In [19]:
ot = LrSinkhornOT()
ot = ot.fit(img_embedding.numpy(),text_embedding.numpy(),np.ones(img_embedding.shape[0])/img_embedding.shape[0],np.ones(text_embedding.shape[0])/text_embedding.shape[0])
res = [test_data.targets[x] for x in ot.P_.argmax(1)]

print(accuracy_score(test_data.targets,res))

0.1024


In [20]:
ot = EntropicOT()
ot = ot.fit(img_embedding.numpy(),text_embedding.numpy(), np.ones(img_embedding.shape[0])/img_embedding.shape[0], np.ones(text_embedding.shape[0])/text_embedding.shape[0])
res = [test_data.targets[x] for x in ot.P_.argmax(1)]

print(accuracy_score(test_data.targets,res))

IndexError: list index out of range

In [21]:
res = [test_data.targets[x] for x in ot.P_.argmax(1)]

print(accuracy_score(test_data.targets,res))

0.1054


In [22]:
ot = InfoOT(num_iters=10)
ot = ot.fit(img_embedding.numpy(),text_embedding.numpy(),np.ones(img_embedding.shape[0])/img_embedding.shape[0],np.ones(text_embedding.shape[0])/text_embedding.shape[0])
res = [test_data.targets[x] for x in ot.P_.argmax(1)]

print(accuracy_score(test_data.targets,res))

In [ ]:
ot = EMD(solver_kwargs={"maxItermax": 10000})
ot = ot.fit(img_embedding.numpy(),text_embedding.numpy(),np.ones(img_embedding.shape[0])/img_embedding.shape[0],np.ones(text_embedding.shape[0])/text_embedding.shape[0])
res = [test_data.targets[x] for x in ot.P_.argmax(1)]

print(accuracy_score(test_data.targets,res))